In [11]:
import pandas as pd
from datetime import datetime
from nose.util import tolist
# 指定数据文件路径
file_path1 = 'HSI_日线数据_with_predict.xlsx'
data = pd.read_excel(file_path1)
file_path2 = 'HSI_5分钟线_with_predict.xlsx'
data1 = pd.read_excel(file_path2)

# 读取从2020-11-17开始的数据
date = tolist(data["日期"])[-737:-1]
close_pre = tolist(data["predict"])[-737:-1]
close = tolist(data["收盘"])[-737:-1]
open_prices = tolist(data["开盘"])[-737:-1]
week = tolist(data['日期'].apply(lambda x:datetime.strptime(x,"%Y-%m-%d").weekday()))

date1 = tolist(data1["日期"])
RSI = tolist(data1["RSI10_5分钟"])
close1 = tolist(data1["收盘"])

# print(date)
# print(close_pre)
# print(close)
# print(open_prices)
# print(week)

# 计算年化收益率与最大回撤率
earn = 0
max_asset = 0  # 最大资产
max_d = 0  # 最大资产回落
day = 0  # 投资天数
num = 0
# 初始资金为100000
init = 1000000
remain = init
# 把初始仓位设为0
position = 0
day_vol=[]

for i in range(3, len(close)):
    # 投资天数加一
    day += 1
    cur_asset = remain + num * open_prices[i]
    day_vol.append(cur_asset)
    if cur_asset > max_asset:
        max_asset = cur_asset
    elif (max_asset - cur_asset) > max_d:
        max_d = max_asset - cur_asset
    
    # 若没有仓位，当天为星期一，且预测值为上涨，则开仓
    # 用预测值做决策，调整参数使收益最大，把决策直接用在实际值上计算收益
    if position == 0 and float(close_pre[i]) > float(close_pre[i - 3]) and week[i] ==0:
        min_RSI = 1
        
        for g in range(1, len(date1)):
            if date1[g] == date[i]:
                cur_RSI = RSI[g] 
                if cur_RSI < min_RSI:
                    min_RSI = cur_RSI
                    min_index = t
                    
        if min_index < 1:
            close_RSI = close1[min_index]
            num = int(remain / close_RSI)
            remain -= close_RSI * num
            position = num * close_RSI / init
        else:  
            num = int(init / float(close[i]))
            remain -= float(close[i]) * num
            position = (init - float(remain)) / init

    # 当涨幅大于10%,平掉所有仓位
    elif position != 0 and float(close_pre[i]) / float(close_pre[i - 1]) >= 1.1:
        max_RSI = 0
        
        for t in range(1, len(date1)):
            if date1[t] == date[i]:
                cur_RSImax = RSI[t]
                if cur_RSImax > max_RSI:
                    max_RSI = cur_RSImax
                    max_index = t
        if max_RSI == 0:
            remain += float(close[i]) * num
            init = remain
            num = 0
            position = 0

        else:
            close_RSI = close1[max_index]
            remain += close_RSI * num
            init = remain
            num = 0
            position = 0
        
    
    # 当跌幅大于0.7%，平掉所有的仓位
    elif position != 0 and (float(close_pre[i - 1]) - float(close_pre[i])) / float(close_pre[i - 1]) > 0.007:
        max_RSI = 0
        
        for t in range(1, len(date1)):
            if date1[t] == date[i]:
                cur_RSImax = RSI[t]
                if cur_RSImax > max_RSI:
                    max_RSI = cur_RSImax
                    max_index = t
        if max_RSI == 0:
            remain += float(close[i]) * num
            init = remain
            num = 0
            position = 0
        else:
            close_RSI = close1[max_index]
            remain += close_RSI * num
            init = remain
            num = 0
            position = 0
        
            
        
    if  date[i] == "2021-05-14" or date[i] == "2021-11-15" or date[i] == "2022-11-14" or date[i] == "2023-11-14":
        print(date[i])
        cur_asset = remain + num * open_prices[i]
        earn = cur_asset - 1000000
        print(earn)
        y_EarnR = (earn / 1000000) / (day / 365)
        print('年化收益率: {:.2%}'.format(y_EarnR))
        maxback = max_d / max_asset
        print('最大回撤率: {:.2%}'.format(maxback))

remain += close[-1] * num
print('最终的结果')
print(remain)
day += 1
earn = remain - 1000000
y_EarnR = (earn / 1000000)/ (day / 365)
print('年化收益率: {:.2%}'.format(y_EarnR))
maxback = max_d / max_asset
print('最大回撤率: {:.2%}'.format(maxback))

2021-05-14
66570.62999999989
年化收益率: 20.42%
最大回撤率: 8.56%
2021-11-15
47858.10999999987
年化收益率: 7.19%
最大回撤率: 12.75%
2022-11-14
36749.600000000326
年化收益率: 2.74%
最大回撤率: 16.11%
最终的结果
1177565.0100000007
年化收益率: 8.83%
最大回撤率: 15.61%
